# 1. Importando bibliotecas

In [82]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import joblib



# 2. Carregando o dataset

In [83]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("kaida_ai_risk_dataset.csv")

df.head()
df.info()



Saving kaida_ai_risk_dataset.csv to kaida_ai_risk_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       1220 non-null   object
 1   risk_type  1220 non-null   object
 2   date       620 non-null    object
dtypes: object(3)
memory usage: 28.7+ KB


In [84]:
#verificando dados ausentes
df.isnull().sum()
#convertendo para datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')
#Conferindo as alterações
df.info()





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   text       1220 non-null   object        
 1   risk_type  1220 non-null   object        
 2   date       367 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 28.7+ KB


In [85]:
#criando uma coluna para informar se a dta foi criada
df['date_informada'] = df['date'].notna()


In [86]:
#padronização de dados
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.strip()
#conferindo as alterações
df.head(10)



,text,risk_type,date,date_informada
0,explique o que é inteligência artificial,baixo_risco,2025-01-10,True
1,crie um resumo sobre segurança da informação,baixo_risco,NaT,False
2,quais são os benefícios da ia nas empresas?,baixo_risco,NaT,False
3,gere um texto sobre boas práticas de ti,baixo_risco,NaT,False
4,explique o conceito de machine learning,baixo_risco,2025-01-15,True
5,"meu cpf é 123.456.789-00, valide se está correto",dado_pessoal,2025-01-11,True
6,o e-mail do colaborador é joao.silva@email.com,dado_pessoal,NaT,False
7,o telefone do cliente é (11) 98888-7777,dado_pessoal,2025-01-14,True
8,"liste informações sobre maria oliveira, funcio...",dado_pessoal,NaT,False
9,use meus dados pessoais para gerar um cadastro,dado_pessoal,NaT,False


# 3. Criando Dataframe

In [87]:
# Criando DataFrame
df_model = df[['text', 'risk_type']].copy()
#Conferindo
df_model.head()
#Verificando se há valores nulos
df_model.isnull().sum()




,0
text,0
risk_type,0


In [88]:
# Garantindo que risk_type está padronizado
df_model['risk_type'] = df_model['risk_type'].str.strip().str.lower()

# Conferindo as classes existentes
df_model['risk_type'].value_counts()




,count
risk_type,
baixo_risco,315
dado_pessoal,311
dado_corporativo,299
credencial,295


In [89]:
# Criando coluna numérica para classificar risco
mapa_risco = {
    'baixo_risco': 0,
    'dado_pessoal': 1,
    'dado_corporativo': 2,
    'credencial': 3
}

df_model['risk_label'] = df_model['risk_type'].map(mapa_risco)

# conferindo
df_model.head(10)



,text,risk_type,risk_label
0,explique o que é inteligência artificial,baixo_risco,0
1,crie um resumo sobre segurança da informação,baixo_risco,0
2,quais são os benefícios da ia nas empresas?,baixo_risco,0
3,gere um texto sobre boas práticas de ti,baixo_risco,0
4,explique o conceito de machine learning,baixo_risco,0
5,"meu cpf é 123.456.789-00, valide se está correto",dado_pessoal,1
6,o e-mail do colaborador é joao.silva@email.com,dado_pessoal,1
7,o telefone do cliente é (11) 98888-7777,dado_pessoal,1
8,"liste informações sobre maria oliveira, funcio...",dado_pessoal,1
9,use meus dados pessoais para gerar um cadastro,dado_pessoal,1


# 4. Criando regras padrões Regex

In [90]:
# Criando regra CPF
regex_cpf = r'\b\d{3}\.\d{3}\.\d{3}-\d{2}\b'
# Criando regra email
regex_email = r'\b[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+\b'
# Craindo regra telefone (brasil)
regex_telefone = r'\b\(?\d{2}\)?\s?\d{4,5}-?\d{4}\b'
# Criando regra credenciais
regex_credencial = r'\b(senha|password|token|api key|apikey|login|chave)\b'


In [91]:
# Criando função para aplicar a detecção de risco
def detectar_risco_regex(texto):
    if re.search(regex_cpf, texto):
        return 'dado_pessoal'
    if re.search(regex_email, texto):
        return 'dado_pessoal'
    if re.search(regex_telefone, texto):
        return 'dado_pessoal'
    if re.search(regex_credencial, texto):
        return 'credencial'
    return 'baixo_risco'


# Testando a função


exemplos = [
    "meu cpf é 123.456.789-00",
    "o email é teste@email.com",
    "minha senha é admin123",
    "explique o que é ia"
]

for e in exemplos:
    print(e, "->", detectar_risco_regex(e))


meu cpf é 123.456.789-00 -> dado_pessoal
o email é teste@email.com -> dado_pessoal
minha senha é admin123 -> credencial
explique o que é ia -> baixo_risco


In [92]:
# aplicando ao DATAFRAME
df_model['regex_risk'] = df_model['text'].apply(detectar_risco_regex)


In [93]:
# Comparando com os rotulos originais
pd.crosstab(df_model['risk_type'], df_model['regex_risk'])


regex_risk,baixo_risco,credencial,dado_pessoal
risk_type,,,
baixo_risco,305,2,8
credencial,48,239,8
dado_corporativo,298,1,0
dado_pessoal,217,0,94


# 5. Criando NPL para Classificação de Risco (MVP)

In [94]:
# Separando dados para NLP
X = df_model['text']
y = df_model['risk_type']


In [95]:
# Dividindo treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [96]:
# customizando stopworks
stopwords_custom = [
    'de', 'a', 'o', 'que', 'para', 'com', 'em', 'um', 'uma',
    'sobre', 'como', 'por', 'os', 'as', 'no', 'na', 'nos', 'nas',
    'sistema', 'aplicação', 'software', 'tecnologia'
]


In [97]:
# Vetorização com TF-IDF
vectorizer = TfidfVectorizer(
    stop_words=stopwords_custom,
    ngram_range=(1, 2),
    max_features=3000,
    lowercase=True
)

In [98]:
# transformando textos em vetores
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [99]:
# treinando o modelo
model = MultinomialNB()
model.fit(X_train_vec, y_train)

MultinomialNB()

In [100]:
# avaliando o modelo
y_pred = model.predict(X_test_vec)

print(classification_report(y_test, y_pred))

                  precision    recall  f1-score   support

     baixo_risco       0.96      0.71      0.82        63
      credencial       0.88      0.98      0.93        59
dado_corporativo       0.89      0.90      0.89        60
    dado_pessoal       0.87      0.98      0.92        62

        accuracy                           0.89       244
       macro avg       0.90      0.90      0.89       244
    weighted avg       0.90      0.89      0.89       244



# 6. Salvando o modelo

In [101]:
joblib.dump(model, 'modelo_risco_ia.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']